In [41]:
import numpy as np
from numpy import linalg as LA
import pandas as pd
from math import radians, cos, sin, asin, sqrt, isnan
import random
from ortools.sat.python import cp_model

In [42]:
#-----------------------------------EXTRACT THE DATA-----------------------------------
requests_path = '../eua-dataset/users/'
U = pd.read_csv(requests_path + 'users-test.csv') # test with 8 users

nodes_path = '../eua-dataset/edge-servers/'
S = pd.read_csv(nodes_path + 'serverstest_v2.csv') # test with 3 servers


In [43]:
#-----------------------------------VARIABLES-----------------------------------
# All the requirements and capacities are randomly generated from a set of possibilites

S_uj = [] # Set of individuals allocated to server s_i


#W = np.array([[1, 2, 1, 2], [2, 3, 3, 4], [5, 7, 6, 6], [1, 2, 1, 2], [2, 3, 3, 4], [5, 7, 6, 6], [1, 2, 1, 2], [2, 3, 3, 4]])
# NORM = []
# for j in range(len(U)):
#     norm = LA.norm(W[j])
#     NORM.append(norm)
W = [2,2,2,2,2,2,2,2]

index = np.argsort(W) # Sort the users by their requirements

# Each server has memory capacity C
#C = np.array([[12,15,15,16],[10,17,15,16],[10,15,11,15]])
C = [16,4,16]
# NORM_C = []
# for i in range(len(S)):
#     norm_c = LA.norm(C[i])
#     NORM_C.append(norm_c)

index_C =np.argsort(C)
index_C = index_C[::-1]


In [44]:
#-----------------------------------HAVERSINE-----------------------------------
def haversine(lon1, lat1, lon2, lat2):
  
    # Convert decimal degrees to radians 
    lon1, lat1, lon2, lat2 = map(radians, [lon1, lat1, lon2, lat2])

    # Haversine formula 
    dlon = lon2 - lon1 
    dlat = lat2 - lat1 
    a = sin(dlat/2)**2 + cos(lat1) * cos(lat2) * sin(dlon/2)**2
    c = 2 * asin(sqrt(a)) 
    r = 6371 # Radius of earth in kilometers. Use 3956 for miles
    return c * r

In [45]:
#-------------------------COVERAGE USER-SERVER--------------------------
#radius = np.round(np.random.uniform(0.1,0.15,len(S)),3) # in km
radius = np.full(len(S), 0.03)

for i in range(len(S)):
  server_latitude = S.iloc[i]['LATITUDE']
  server_longitude = S.iloc[i]['LONGITUDE']
  temp = []
  for j in range(len(U)):
    user_latitude = U.iloc[j]['Latitude']
    user_longitude = U.iloc[j]['Longitude']

    dist_geo = haversine(server_longitude, server_latitude, user_longitude, user_latitude)
    
    if dist_geo <= radius[i]:
        temp.append(1)
        
    else:
        temp.append(0)
       
  S_uj.append(temp)

In [46]:
#-------------------------Decision variables--------------
x_ij = np.zeros(shape=(len(S),len(U)))
y_i = np.zeros(len(S))
S_active = np.full(len(S),0)

In [47]:
# user_index=0
# while user_index in index:
#     j= index[user_index]
#     print("USER:  ",j)
#     if all(S_active==0):
#         print("---------NON ACTIVE----------------------")
#         for i in index_C:       
#             if all(W[j]<C[i]) and S_uj[i][j]==1:
#                 print("ENTRO j: ",j,"i: ",i)
#                 C[i]=C[i]-W[j]
#                 S_active[i]=1
#                 y_i[j]=1
#                 x_ij[i][j]=1
#                 user_index=user_index+1           
#     elif any(S_active==1):
#         print("---------AT LEAST ONE ACTIVE----------------------")
#         active_loc=np.where(S_active==1)[0][:]
#         l=0
#         for i in index_C:
#             if all(W[j]<C[i]) and S_uj[i][j]==1 and (i in active_loc):
#                 print("ENTRO j: ",i,"i: ",j)
#                 C[i]=C[i]-W[j]
#                 y_i[i]=1
#                 x_ij[i][j]=1
#                 l=1
#                 user_index=user_index+1 
#         if l==0:
#             for i in index_C:
#                 if all(W[j]<C[i]) and S_uj[i][j]==1:
#                     C[i]=C[i]-W[j]
#                     S_active[i]=1
#                     y_i[i]=1
#                     x_ij[i][j]=1
#                     user_index=user_index+1 
#     user_index=user_index+1 

In [48]:
user_index=0
while user_index in index:
    j= index[user_index]
    index_C =np.argsort(C)
    index_C = index_C[::-1]
    loc =0
    print("---------------------USER:  ",j," ----------------------------------")
    print("ORDER INDEX C", index_C)
    if all(S_active==0):
        print("---------NON ACTIVE---------")
        for i in index_C:   
            print("i:",i)    
            #if all(W[j]<C[i]) and S_uj[i][j]==1:
            if W[j]<=C[i] and S_uj[i][j]==1:
                
                C[i]=C[i]-W[j]
                S_active[i]=1
                y_i[i]=1
                x_ij[i][j]=1
                print("ENTRO j: ",j,"i: ",i)
                print("Value C: ", C)
                break
    
    elif any(S_active==1) and loc==0:
        print("---------AT LEAST ONE ACTIVE---------")
        active_loc=np.where(S_active==1)[0][:]
        print("possible servers: ", active_loc)
        for i_act in index_C:
            #if all(W[j]<C[i_act]) and S_uj[i_act][j]==1:
            if W[j]<=C[i_act] and S_uj[i_act][j]==1 and (i_act in active_loc) and loc==0:
                C[i_act]=C[i_act]-W[j]
                y_i[i_act]=1
                x_ij[i_act][j]=1
                loc=1
                print("ENTRO EXISTING SERVER:",i_act,"ENTRO j: ",i,"i: ",j)
                print("Value C: ", C)
                break
        for i in index_C:
            #if all(W[j]<C[i]) and S_uj[i][j]==1 and (i not in active_loc):
            if W[j]<=C[i] and S_uj[i][j]==1 and (i not in active_loc) and loc==0:
                C[i]=C[i]-W[j]
                S_active[i]=1
                y_i[i]=1
                x_ij[i][j]=1
                loc=1
                print("ACTIVATE NEW SERVER: ",i,"ENTRO j: ",i,"i: ",j)
                print("Value C: ",C)
                break
    user_index=user_index+1 

---------------------USER:   0  ----------------------------------
ORDER INDEX C [2 0 1]
---------NON ACTIVE---------
i: 2
ENTRO j:  0 i:  2
Value C:  [16, 4, 14]
---------------------USER:   1  ----------------------------------
ORDER INDEX C [0 2 1]
---------AT LEAST ONE ACTIVE---------
possible servers:  [2]
ACTIVATE NEW SERVER:  0 ENTRO j:  0 i:  1
Value C:  [14, 4, 14]
---------------------USER:   2  ----------------------------------
ORDER INDEX C [2 0 1]
---------AT LEAST ONE ACTIVE---------
possible servers:  [0 2]
ENTRO EXISTING SERVER: 2 ENTRO j:  0 i:  2
Value C:  [14, 4, 12]
---------------------USER:   3  ----------------------------------
ORDER INDEX C [0 2 1]
---------AT LEAST ONE ACTIVE---------
possible servers:  [0 2]
ACTIVATE NEW SERVER:  1 ENTRO j:  1 i:  3
Value C:  [14, 2, 12]
---------------------USER:   4  ----------------------------------
ORDER INDEX C [0 2 1]
---------AT LEAST ONE ACTIVE---------
possible servers:  [0 1 2]
ENTRO EXISTING SERVER: 1 ENTRO j:  1

In [49]:
W[0]<C[0]

True

In [50]:
p=[2,10,5]
for i in p:
    print(i)

2
10
5
